In [1]:
import sys

sys.path.insert(0, '..')
import os
from biosimulator_processes.process_bigraph import pp, pf
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

CobraProcess registered successfully.
CopasiProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


True

In [2]:
class P:
    def __init__(self,
                 config=None):
        self.config = config
        if isinstance(self.config, dict):
            self.model = TimeCourseModel(**self.config['model'])

In [3]:
model = {
    'model': {
        'model_source': model_filepath
    }
}


tc = P(config=model)


In [4]:
pp(tc.model.model_dump()), type(tc.model.model_source)

{ 'model_changes': None,
  'model_id': '',
  'model_language': 'sbml',
  'model_name': 'Unnamed Composite Process TimeCourseModel',
  'model_source': { 'value': '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'},
  'model_units': '_default'}


(None, biosimulator_processes.data_model.ModelFilepath)

In [5]:
tc.model_id

AttributeError: 'P' object has no attribute 'model_id'

In [ ]:
x = {'a': '234', 'b': 20}

p = ProcessConfig(**x)

In [6]:
from biosimulator_processes.data_model import dynamic_process_config as DPC


config = DPC(name='rose', age=29)

PydanticUserError: A non-annotated attribute was detected: `field_definitions = {'name': 'rose', 'age': 29}`. All model fields require a type annotation; if `field_definitions` is not meant to be a field, you may be able to resolve this error by annotating it as a `ClassVar` or updating `model_config['ignored_types']`.

For further information visit https://errors.pydantic.dev/2.6/u/model-field-missing-annotation

In [7]:
from pydantic import create_model


def dynamic_config(name: str, config: dict, **kwargs):
    config = config or {}
    config.update(kwargs)
    dynamic_config_types = {}
    for param_name, param_val in config.items():
        dynamic_config_types[param_name] = (type(param_val), ...)

    name = name.replace(name[0], name[0].upper())
    DynamicProcessConfig = create_model(
            f'{name}ProcessConfig',
            **dynamic_config_types
    )
    return DynamicProcessConfig(**config)

In [ ]:
x = [1, 2, 3]

y = tuple(x)

In [ ]:
y